### Importing

In [2]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

air_canada = pd.read_csv('../datasets/participant_data.csv')

# seperate dataset into training and predicting
air_canada_to_predict = air_canada[air_canada['choice'] != air_canada['choice']]
air_canada = air_canada[air_canada['choice'] == air_canada['choice']]

X, y = air_canada.drop(['id', 'ticket_id', 'choice'], axis=1), air_canada[['choice']]

y['choice'] = y['choice'].map({ 'nochoice': 0, 'pref': 1, 'advs': 2 })

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

/tmp/ipykernel_150129/3560916056.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y['choice'] = y['choice'].map({ 'nochoice': 0, 'pref': 1, 'advs': 2 })


In [3]:
y

,choice
2,1
3,0
4,1
7,1
8,0
...,...
256657,0
256658,0
256660,2
256661,2


In [4]:
air_canada.sample(5)

,id,ticket_id,od,flight_departure_datetime,purchase_datetime,trip_type,branded_fare,number_of_pax,ADVS_price,PREF_price,ADVS_capacity,PREF_capacity,ADVS_inventory,PREF_inventory,choice
164888,164888,8624458990387683,CCCDDD,2024-08-22 19:00,2024-07-01 16:44,0,1,1,37,69,250,50,237,25,pref
103234,103234,1398636002513264,AAADDD,2024-05-26 22:00,2024-05-06 12:47,0,0,3,62,99,110,30,102,30,nochoice
35351,35351,"7,70E+15",BBBCCC,2024-02-20 10:00,2024-02-17 02:46,0,1,1,59,94,250,50,250,49,nochoice
179367,179367,99213115646188,AAADDD,2024-09-11 22:00,2024-08-12 19:27,0,1,2,51,118,110,30,101,23,nochoice
225545,225545,8506909166236544,DDDEEE,2024-11-16 05:00,2024-10-08 22:47,1,0,3,49,91,110,30,99,23,nochoice


In [5]:
X.columns

Index(['od', 'flight_departure_datetime', 'purchase_datetime', 'trip_type',
       'branded_fare', 'number_of_pax', 'ADVS_price', 'PREF_price',
       'ADVS_capacity', 'PREF_capacity', 'ADVS_inventory', 'PREF_inventory'],
      dtype='object')

### Preprocessing

In [6]:
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

In [7]:
num_pipeline = make_pipeline(
  StandardScaler()
)

cat_pipeline = make_pipeline(
  OneHotEncoder()
)
preprocessing = ColumnTransformer([
    ('cat', cat_pipeline, ['od']),
    ('num', num_pipeline, ['trip_type', 'branded_fare', 'number_of_pax', 'ADVS_price', 'PREF_price',
       'ADVS_capacity', 'PREF_capacity', 'ADVS_inventory', 'PREF_inventory'])
  ], remainder='drop')

In [8]:
X_train_prep = preprocessing.fit_transform(X_train)
X_test_prep = preprocessing.transform(X_test)

In [9]:
X_train_prep[0]

array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  1.        ,  0.        ,
       -0.50803601,  0.99980736,  0.28432061, -0.57887917,  0.21665309,
        0.62208764,  0.5888285 ,  0.98514872,  0.90658342])

### Training

In [10]:
from xgboost import XGBClassifier

bst = XGBClassifier(learning_rate=.3, objective='multi:softprob')
bst.fit(X_train_prep, y_train)


y_pred = bst.predict(X_test_prep)
accuracy_score(y_pred, y_test)

0.7495584415584415

### Tuning

In [12]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [100, 200, 300],           # Number of boosting rounds
    'learning_rate': [0.01, 0.1, 0.2],         # Step size shrinkage
    'max_depth': [3, 5, 7],                    # Maximum depth of the trees
    'min_child_weight': [1, 3, 5],             # Minimum sum of instance weight needed in a child
    'subsample': [0.6, 0.8, 1.0],              # Subsample ratio of the training instances
    'colsample_bytree': [0.6, 0.8, 1.0],       # Subsample ratio of columns when constructing each tree
    'gamma': [0, 0.1, 0.2],                    # Minimum loss reduction to make a further partition on a leaf node
}

rnd_search = GridSearchCV(bst,
    param_grid=param_grid, verbose=3, cv=3, n_jobs=-1,
    scoring='neg_root_mean_squared_error')

rnd_search.fit(X_train_prep, y_train)

Fitting 3 folds for each of 2187 candidates, totalling 6561 fits


KeyboardInterrupt: 

### Predicting

In [33]:
X_to_predict, y_to_predict = air_canada_to_predict.drop(['id', 'ticket_id', 'choice'], axis=1), air_canada_to_predict[['id']]
X_to_predict_prep = preprocessing.transform(X_to_predict)
y_to_predict['choice'] = bst.predict(X_to_predict_prep)

/tmp/ipykernel_150129/1900845532.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_to_predict['choice'] = bst.predict(X_to_predict_prep)


In [41]:
y_to_predict['choice'] = y_to_predict['choice'].map({ 0: 'nochoice', 1: 'pref', 2: 'advs'})

/tmp/ipykernel_150129/1816271922.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_to_predict['choice'] = y_to_predict['choice'].map({ 0: 'nochoice', 1: 'pref', 2: 'advs'})


In [45]:
y_final = pd.concat([y_to_predict, air_canada])[['id', 'choice']]

### Saving

In [46]:
pd.DataFrame.to_csv(y_final, '../predictions/pred.csv', index=False)